In [2]:
import os
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
import torch

In [3]:
from dotenv import load_dotenv

In [4]:
import os
from langchain_huggingface import HuggingFacePipeline 
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
import torch
from dotenv import load_dotenv

# Set HuggingFace token
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")

# Set device
device = 0 if torch.cuda.is_available() else -1

# Initialize tokenizer first
model_id = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ["HF_TOKEN"])

# Set up padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Initialize model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=os.environ["HF_TOKEN"],
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto" if torch.cuda.is_available() else None
)

# Create the pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500,
    min_new_tokens=30,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True,
    padding=True
)

# Initialize the LangChain pipeline
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create a prompt template
prompt = PromptTemplate.from_template("""Question: {question}
Answer: Let me provide a detailed response:""")

# Create output parser
output_parser = StrOutputParser()

# Create the chain
chain = prompt | llm | output_parser

# Directory to save the model
save_directory = "F:\LLM\model"  # Update this to your desired path

# Save model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

# Run the chain with error handling
try:
    question = "My name is Clara and I am"
    response = chain.invoke({"question": question})
    print(response)
except Exception as e:
    print(f"Error during inference: {str(e)}")

Some parameters are on the meta device because they were offloaded to the cpu.
c:\Users\shrey\.conda\envs\gen_ai\Lib\site-packages\transformers\modeling_utils.py:2670: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/1 [00:00<?, ?it/s]

SafetensorError: Error while serializing: IoError(Os { code: 1224, kind: Uncategorized, message: "The requested operation cannot be performed on a file with a user-mapped section open." })

In [1]:
question = "My name is Shreyas and I am"
response = chain.invoke({"question": question})
print(response)

NameError: name 'chain' is not defined

In [8]:
response = llm("My name is Clara and I am")

In [10]:
print(response)

My name is Clara and I am a freelance illustrator and I live in Denmark. I love creating all kinds of illustrations, whether it is a digital illustration, a book cover, or a poster. I'm always looking for new challenges and new ways to tell stories through my work.
